# Predict Digit

In [68]:
# predict_digit_dataset_improved.py
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import os

# --- Configuration ---
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 # Increased epochs since we use Early Stopping
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("predict_digit_dataset.csv")  # CSV you created
print("Dataset loaded:", df.shape)

# ------------------------
## Define Features and Target
# ------------------------
# This model only uses sensor data (12 features) to predict the current digit.
sensor_cols = ['accX','accY','accZ',
               'gyroX','gyroY','gyroZ',
               'rotX','rotY','rotZ',
               'magX','magY','magZ']

X_sensor = df[sensor_cols].astype(float)
y = LabelEncoder().fit_transform(df['digit']) # Target is the raw digit index (0-9)

# Scale sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sensor)

# ------------------------
## Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

# ------------------------
## Build Model (with Regularization)
# ------------------------
num_classes = len(np.unique(y))

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    # Use sparse_categorical_crossentropy since y is integer-encoded (0, 1, 2, ...)
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', # Stop when validation loss stops improving
    patience=10,        # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

print(f"Training for {num_classes} classes (Input size: {X_train.shape[1]})...")

history = model.fit(
    X_train, y_train, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop] # Apply Early Stopping
)

# ------------------------
## Evaluate & Save
# ------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {acc:.4f}")

# Save Keras model
os.makedirs("models_tflite", exist_ok=True)
keras_path = "models_tflite/predict_digit_model.h5"
model.save(keras_path)
print(f"Keras model saved to: {keras_path}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "predict_digit_model.tflite" # Output path kept for direct app use
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_path}")

Dataset loaded: (17675, 13)
Training for 10 classes (Input size: 12)...
Epoch 1/100
707/707 [==============================] - 1s 680us/step - loss: 1.8692 - accuracy: 0.2996 - val_loss: 1.5415 - val_accuracy: 0.4325
Epoch 2/100
707/707 [==============================] - 0s 614us/step - loss: 1.5685 - accuracy: 0.4130 - val_loss: 1.3767 - val_accuracy: 0.5000
Epoch 3/100
707/707 [==============================] - 0s 605us/step - loss: 1.4405 - accuracy: 0.4759 - val_loss: 1.2557 - val_accuracy: 0.5941
Epoch 4/100
707/707 [==============================] - 0s 608us/step - loss: 1.3384 - accuracy: 0.5260 - val_loss: 1.1554 - val_accuracy: 0.6234
Epoch 5/100
707/707 [==============================] - 0s 602us/step - loss: 1.2701 - accuracy: 0.5560 - val_loss: 1.0719 - val_accuracy: 0.6768
Epoch 6/100
707/707 [==============================] - 0s 616us/step - loss: 1.1964 - accuracy: 0.5927 - val_loss: 1.0052 - val_accuracy: 0.7044
Epoch 7/100
707/707 [==============================] - 0s 

/Users/hmanasi1/anaconda3/envs/iot-project-env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp92vjnzqj/assets


TFLite model saved to: predict_digit_model.tflite


2025-11-30 18:31:33.806342: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:31:33.806353: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:31:33.806429: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp92vjnzqj
2025-11-30 18:31:33.806966: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:31:33.806969: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp92vjnzqj
2025-11-30 18:31:33.808257: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:31:33.823630: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp92vjnzqj
2025-11-

# Predict 2nd Digit

In [69]:
# predict_2nd_digit_dataset_improved_weighted.py
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import os

# --- Configuration ---
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
NUM_CLASSES = 10 
DIGIT_WEIGHT_FACTOR = 5.0 # Explicit weight to apply to OHE features
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("predict_2nd_digit.csv")
print("Dataset loaded:", df.shape)

# ------------------------
## Define Features and Target
# ------------------------
digit_col = ['digit1']
sensor_cols = ['acc1X','acc1Y','acc1Z',
               'gyro1X','gyro1Y','gyro1Z',
               'rot1X','rot1Y','rot1Z',
               'mag1X','mag1Y','mag1Z']

# 1. One-Hot Encode Digit Feature
# Input size: 10 features
X_digits = pd.get_dummies(df[digit_col].astype(str)) 

# 2. Apply Weight Factor to OHE Features
X_digits_weighted = X_digits * DIGIT_WEIGHT_FACTOR # <-- Weighting Applied

# 3. Scale Sensor Features
# Input size: 12 features
X_sensors = df[sensor_cols].astype(float)
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors)
X_sensors_scaled_df = pd.DataFrame(X_sensors_scaled, columns=sensor_cols, index=df.index)

# 4. Concatenate Features
# Final feature size: 10 (OHE * 5) + 12 (Scaled Sensors) = 22 features
X = pd.concat([X_digits_weighted, X_sensors_scaled_df], axis=1) 
X_array = X.values.astype(np.float32)

# Target: digit2 is the target, label encoded
y = LabelEncoder().fit_transform(df['digit2'])

# ------------------------
## Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_array, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

# ------------------------
## Build Model (with Regularization)
# ------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (10 classes for digit prediction)
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Training for {NUM_CLASSES} classes (Input size: {X_train.shape[1]})...")

model.fit(
    X_train, y_train, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Evaluate & Save
# ------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {acc:.4f}")

# Save Keras model
os.makedirs("models_tflite", exist_ok=True)
keras_path = "models_tflite/predict_2nd_digit_model.h5"
model.save(keras_path)
print(f"Keras model saved to: {keras_path}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "predict_2nd_digit_model.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_path}")

Dataset loaded: (5050, 14)
Training for 10 classes (Input size: 22)...
Epoch 1/100
202/202 [==============================] - 0s 838us/step - loss: 1.7626 - accuracy: 0.4155 - val_loss: 1.0362 - val_accuracy: 0.7500
Epoch 2/100
202/202 [==============================] - 0s 646us/step - loss: 1.0095 - accuracy: 0.6838 - val_loss: 0.5784 - val_accuracy: 0.8998
Epoch 3/100
202/202 [==============================] - 0s 630us/step - loss: 0.6970 - accuracy: 0.7942 - val_loss: 0.3900 - val_accuracy: 0.9554
Epoch 4/100
202/202 [==============================] - 0s 627us/step - loss: 0.5303 - accuracy: 0.8713 - val_loss: 0.3242 - val_accuracy: 0.9579
Epoch 5/100
202/202 [==============================] - 0s 645us/step - loss: 0.4326 - accuracy: 0.9041 - val_loss: 0.2707 - val_accuracy: 0.9703
Epoch 6/100
202/202 [==============================] - 0s 638us/step - loss: 0.3888 - accuracy: 0.9205 - val_loss: 0.2488 - val_accuracy: 0.9728
Epoch 7/100
202/202 [==============================] - 0s 6

/Users/hmanasi1/anaconda3/envs/iot-project-env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpsy3q1krz/assets


TFLite model saved to: predict_2nd_digit_model.tflite


2025-11-30 18:31:47.414034: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:31:47.414045: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:31:47.414126: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpsy3q1krz
2025-11-30 18:31:47.414607: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:31:47.414611: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpsy3q1krz
2025-11-30 18:31:47.415732: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:31:47.431046: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpsy3q1krz
2025-11-

# Predict 3rd Digit

In [70]:
# predict_3rd_digit_model_improved_weighted.py
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import os

# --- Configuration ---
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
NUM_CLASSES = 10 # Predicting the next digit (0-9)
DIGIT_WEIGHT_FACTOR = 5.0 # Explicit weight to apply to OHE features
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("predict_3rd_digit.csv")
print("Dataset loaded:", df.shape)

# ------------------------
## Define Features and Target
# ------------------------
digit_cols = ['digit1', 'digit2']
sensor_cols = ['acc1X','acc1Y','acc1Z','gyro1X','gyro1Y','gyro1Z','rot1X','rot1Y','rot1Z','mag1X','mag1Y','mag1Z',
               'acc2X','acc2Y','acc2Z','gyro2X','gyro2Y','gyro2Z','rot2X','rot2Y','rot2Z','mag2X','mag2Y','mag2Z']

# 1. One-Hot Encode Digit Features
# Input size: 20 features (10 for digit1, 10 for digit2)
X_digits = pd.get_dummies(df[digit_cols].astype(str)) 

# 2. Apply Weight Factor to OHE Features
X_digits_weighted = X_digits * DIGIT_WEIGHT_FACTOR # <-- Weighting Applied

# 3. Scale Sensor Features
# Input size: 24 features
X_sensors = df[sensor_cols].astype(float)
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors)
X_sensors_scaled_df = pd.DataFrame(X_sensors_scaled, columns=sensor_cols, index=df.index)

# 4. Concatenate Features
# Final feature size: 20 (OHE * 5) + 24 (Scaled Sensors) = 44 features
X = pd.concat([X_digits_weighted, X_sensors_scaled_df], axis=1) 
X_array = X.values.astype(np.float32)

# Target: digit3 is the target, label encoded
y = LabelEncoder().fit_transform(df['digit3'])

# ------------------------
## Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_array, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

# ------------------------
## Build Model (with Regularization)
# ------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (10 classes for digit prediction)
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Training for {NUM_CLASSES} classes (Input size: {X_train.shape[1]})...")

model.fit(
    X_train, y_train, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Evaluate & Save
# ------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {acc:.4f}")

# Save Keras model
os.makedirs("models_tflite", exist_ok=True)
keras_path = "models_tflite/predict_3rd_digit_model.h5"
model.save(keras_path)
print(f"Keras model saved to: {keras_path}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "predict_3rd_digit_model.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_path}")

Dataset loaded: (5050, 27)
Training for 10 classes (Input size: 44)...
Epoch 1/100
202/202 [==============================] - 0s 844us/step - loss: 1.4649 - accuracy: 0.5699 - val_loss: 0.3900 - val_accuracy: 0.9814
Epoch 2/100
202/202 [==============================] - 0s 652us/step - loss: 0.4447 - accuracy: 0.9313 - val_loss: 0.2051 - val_accuracy: 0.9938
Epoch 3/100
202/202 [==============================] - 0s 637us/step - loss: 0.2989 - accuracy: 0.9663 - val_loss: 0.1754 - val_accuracy: 1.0000
Epoch 4/100
202/202 [==============================] - 0s 647us/step - loss: 0.2341 - accuracy: 0.9876 - val_loss: 0.1646 - val_accuracy: 1.0000
Epoch 5/100
202/202 [==============================] - 0s 663us/step - loss: 0.2043 - accuracy: 0.9916 - val_loss: 0.1561 - val_accuracy: 1.0000
Epoch 6/100
202/202 [==============================] - 0s 647us/step - loss: 0.1933 - accuracy: 0.9920 - val_loss: 0.1488 - val_accuracy: 1.0000
Epoch 7/100
202/202 [==============================] - 0s 6

/Users/hmanasi1/anaconda3/envs/iot-project-env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmphayfytbo/assets


TFLite model saved to: predict_3rd_digit_model.tflite


2025-11-30 18:32:00.603650: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:00.603661: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:00.603739: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmphayfytbo
2025-11-30 18:32:00.604236: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:00.604239: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmphayfytbo
2025-11-30 18:32:00.605506: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:00.620604: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmphayfytbo
2025-11-

# Predict 4th Digit

In [71]:
# predict_4th_digit_model_improved_weighted.py
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import os

# --- Configuration ---
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
NUM_CLASSES = 10 # Predicting the next digit (0-9)
DIGIT_WEIGHT_FACTOR = 5.0 # Explicit weight to apply to OHE features
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("predict_4th_digit.csv")
print("Dataset loaded:", df.shape)

# ------------------------
## Define Features and Target
# ------------------------
digit_cols = ['digit1', 'digit2', 'digit3']
sensor_cols = [
    'acc1X','acc1Y','acc1Z','gyro1X','gyro1Y','gyro1Z','rot1X','rot1Y','rot1Z','mag1X','mag1Y','mag1Z',
    'acc2X','acc2Y','acc2Z','gyro2X','gyro2Y','gyro2Z','rot2X','rot2Y','rot2Z','mag2X','mag2Y','mag2Z',
    'acc3X','acc3Y','acc3Z','gyro3X','gyro3Y','gyro3Z','rot3X','rot3Y','rot3Z','mag3X','mag3Y','mag3Z'
]

# 1. One-Hot Encode Digit Features
# Input size: 30 features (10 for each digit)
X_digits = pd.get_dummies(df[digit_cols].astype(str)) 

# 2. Apply Weight Factor to OHE Features
X_digits_weighted = X_digits * DIGIT_WEIGHT_FACTOR # <-- Weighting Applied

# 3. Scale Sensor Features
# Input size: 36 features
X_sensors = df[sensor_cols].astype(float)
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors)
X_sensors_scaled_df = pd.DataFrame(X_sensors_scaled, columns=sensor_cols, index=df.index)

# 4. Concatenate Features
# Final feature size: 30 (OHE * 5) + 36 (Scaled Sensors) = 66 features
X = pd.concat([X_digits_weighted, X_sensors_scaled_df], axis=1) 
X_array = X.values.astype(np.float32)

# Target: digit4 is the target, label encoded
y = LabelEncoder().fit_transform(df['digit4'])

# ------------------------
## Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_array, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

# ------------------------
## Build Model (with Regularization)
# ------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (10 classes for digit prediction)
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Training for {NUM_CLASSES} classes (Input size: {X_train.shape[1]})...")

model.fit(
    X_train, y_train, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Evaluate & Save
# ------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {acc:.4f}")

# Save Keras model
os.makedirs("models_tflite", exist_ok=True)
keras_path = "models_tflite/predict_4th_digit_model.h5"
model.save(keras_path)
print(f"Keras model saved to: {keras_path}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "predict_4th_digit_model.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_path}")

Dataset loaded: (2525, 40)
Training for 10 classes (Input size: 66)...
Epoch 1/100
101/101 [==============================] - 0s 1ms/step - loss: 1.2946 - accuracy: 0.6380 - val_loss: 0.2676 - val_accuracy: 1.0000
Epoch 2/100
101/101 [==============================] - 0s 716us/step - loss: 0.3544 - accuracy: 0.9672 - val_loss: 0.1807 - val_accuracy: 1.0000
Epoch 3/100
101/101 [==============================] - 0s 679us/step - loss: 0.2323 - accuracy: 0.9926 - val_loss: 0.1713 - val_accuracy: 1.0000
Epoch 4/100
101/101 [==============================] - 0s 698us/step - loss: 0.2027 - accuracy: 0.9975 - val_loss: 0.1657 - val_accuracy: 1.0000
Epoch 5/100
101/101 [==============================] - 0s 701us/step - loss: 0.1864 - accuracy: 0.9981 - val_loss: 0.1600 - val_accuracy: 1.0000
Epoch 6/100
101/101 [==============================] - 0s 695us/step - loss: 0.1719 - accuracy: 1.0000 - val_loss: 0.1536 - val_accuracy: 1.0000
Epoch 7/100
101/101 [==============================] - 0s 695

/Users/hmanasi1/anaconda3/envs/iot-project-env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6y_qhmub/assets


TFLite model saved to: predict_4th_digit_model.tflite


2025-11-30 18:32:08.154782: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:08.154793: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:08.154881: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6y_qhmub
2025-11-30 18:32:08.155478: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:08.155482: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6y_qhmub
2025-11-30 18:32:08.156764: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:08.172412: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6y_qhmub
2025-11-

# Predict 3 digit sequences

## Using 1 digit

In [72]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("3digit_predict_seq_after_1.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features (digit1)
digit_data = feature_X['digit1'].values.astype(int)
X_digit_ohe = to_categorical(digit_data, num_classes=NUM_DIGITS) # 10 OHE features

# 3. Extract and Scale Sensor Features
# The sensor data starts from the second column (index 1) in the feature_X DataFrame
X_sensors_raw = feature_X.drop(columns=['digit1']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw)

# 4. Concatenate OHE digits and Scaled Sensors
# The final feature matrix X now has (10 OHE features + 12 sensor features) = 22 features
X = np.concatenate([X_digit_ohe, X_sensors_scaled], axis=1)


# Target Encoding (Predicting the full sequence ID)
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 22
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (num_classes, likely 25)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (10 OHE + 12 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=0.2, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_3digit_seq_after_1.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 3-digit sequence after 1st digit saved to {tflite_path}")

Dataset loaded: (2525, 14)
Final Model Input Shape: 22 (10 OHE + 12 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 985us/step - loss: 2.3109 - accuracy: 0.4059 - val_loss: 5.8514 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 697us/step - loss: 0.9011 - accuracy: 0.8272 - val_loss: 9.0972 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 686us/step - loss: 0.5200 - accuracy: 0.8985 - val_loss: 10.3012 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 683us/step - loss: 0.3954 - accuracy: 0.9371 - val_loss: 11.1010 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 688us/step - loss: 0.3508 - accuracy: 0.9470 - val_loss: 11.6878 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 685us/step - loss: 0.3089 - accuracy: 0.9629 - val_loss: 12.3452 - val_accuracy: 0.0000e+00


INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpbdgtd4yj/assets


TFLite model for 3-digit sequence after 1st digit saved to tflite_3digit_seq_after_1.tflite


2025-11-30 18:32:09.596077: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:09.596090: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:09.596175: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpbdgtd4yj
2025-11-30 18:32:09.596723: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:09.596727: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpbdgtd4yj
2025-11-30 18:32:09.597965: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:09.613459: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpbdgtd4yj
2025-11-

## Using 2 digits

In [73]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("3digit_predict_seq_after_2.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features
digit1_data = feature_X['digit1'].values.astype(int)
digit2_data = feature_X['digit2'].values.astype(int)

X_digit1_ohe = to_categorical(digit1_data, num_classes=NUM_DIGITS) # 10 features
X_digit2_ohe = to_categorical(digit2_data, num_classes=NUM_DIGITS) # 10 features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1', 'digit2']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw) # 24 sensor features (12 * 2 snapshots)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (10 + 10 + 24) = 44 features
X = np.concatenate([X_digit1_ohe, X_digit2_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 44
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (20 OHE + 24 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_3digit_seq_after_2.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 3-digit sequence after 2nd digit saved to {tflite_path}")

Dataset loaded: (2525, 27)
Final Model Input Shape: 44 (20 OHE + 24 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 998us/step - loss: 2.1241 - accuracy: 0.5129 - val_loss: 5.8263 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 705us/step - loss: 0.5841 - accuracy: 0.9391 - val_loss: 8.5601 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 695us/step - loss: 0.3146 - accuracy: 0.9837 - val_loss: 9.5794 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 669us/step - loss: 0.2561 - accuracy: 0.9916 - val_loss: 10.0119 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 678us/step - loss: 0.2319 - accuracy: 0.9891 - val_loss: 10.3140 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 687us/step - loss: 0.2139 - accuracy: 0.9950 - val_loss: 10.3825 - val_accuracy: 0.0000e+00
E

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp19xrt09q/assets


TFLite model for 3-digit sequence after 2nd digit saved to tflite_3digit_seq_after_2.tflite


2025-11-30 18:32:11.042997: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:11.043012: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:11.043091: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp19xrt09q
2025-11-30 18:32:11.043647: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:11.043650: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp19xrt09q
2025-11-30 18:32:11.044904: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:11.060238: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp19xrt09q
2025-11-

## Using 3 digits

In [74]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("3digit_predict_seq_after_3.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features
digit1_data = feature_X['digit1'].values.astype(int)
digit2_data = feature_X['digit2'].values.astype(int)
digit3_data = feature_X['digit3'].values.astype(int)

X_digit1_ohe = to_categorical(digit1_data, num_classes=NUM_DIGITS) # 10 features
X_digit2_ohe = to_categorical(digit2_data, num_classes=NUM_DIGITS) # 10 features
X_digit3_ohe = to_categorical(digit3_data, num_classes=NUM_DIGITS) # 10 features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1', 'digit2', 'digit3']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw) # 36 sensor features (12 * 3 snapshots)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (10 + 10 + 10 + 36) = 66 features
X = np.concatenate([X_digit1_ohe, X_digit2_ohe, X_digit3_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 66
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (30 OHE + 36 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_3digit_seq_after_3.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 3-digit sequence after 3rd digit saved to {tflite_path}")

Dataset loaded: (2525, 40)
Final Model Input Shape: 66 (30 OHE + 36 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 975us/step - loss: 1.8594 - accuracy: 0.5970 - val_loss: 7.0676 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 730us/step - loss: 0.4510 - accuracy: 0.9693 - val_loss: 9.1148 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 711us/step - loss: 0.2881 - accuracy: 0.9881 - val_loss: 9.8115 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 699us/step - loss: 0.2349 - accuracy: 0.9965 - val_loss: 10.0559 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 698us/step - loss: 0.2180 - accuracy: 0.9995 - val_loss: 10.2337 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 726us/step - loss: 0.2003 - accuracy: 0.9975 - val_loss: 10.4377 - val_accuracy: 0.0000e+00
E

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp8kg1v8bs/assets


TFLite model for 3-digit sequence after 3rd digit saved to tflite_3digit_seq_after_3.tflite


2025-11-30 18:32:12.663192: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:12.663203: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:12.663285: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp8kg1v8bs
2025-11-30 18:32:12.663938: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:12.663942: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp8kg1v8bs
2025-11-30 18:32:12.665283: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:12.680904: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp8kg1v8bs
2025-11-

# Predict 4 digit sequences

## Using 1 digit

In [75]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("4digit_predict_seq_after_1.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features (digit1)
digit_data = feature_X['digit1'].values.astype(int)
X_digit_ohe = to_categorical(digit_data, num_classes=NUM_DIGITS) # 10 OHE features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (10 OHE features + 12 sensor features) = 22 features
X = np.concatenate([X_digit_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 22
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (10 OHE + 12 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_4digit_seq_after_1.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 4-digit sequence after 1st digit saved to {tflite_path}")

Dataset loaded: (2525, 14)
Final Model Input Shape: 22 (10 OHE + 12 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 1ms/step - loss: 2.4599 - accuracy: 0.3500 - val_loss: 6.0671 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 762us/step - loss: 1.0059 - accuracy: 0.8000 - val_loss: 8.8922 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 724us/step - loss: 0.5214 - accuracy: 0.9233 - val_loss: 10.0551 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 712us/step - loss: 0.3592 - accuracy: 0.9668 - val_loss: 10.9076 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 714us/step - loss: 0.3135 - accuracy: 0.9772 - val_loss: 11.5605 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 725us/step - loss: 0.2787 - accuracy: 0.9822 - val_loss: 11.7912 - val_accuracy: 0.0000e+00
Ep

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpgk1g058t/assets


TFLite model for 4-digit sequence after 1st digit saved to tflite_4digit_seq_after_1.tflite


2025-11-30 18:32:14.146473: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:14.146484: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:14.146564: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpgk1g058t
2025-11-30 18:32:14.147112: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:14.147116: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpgk1g058t
2025-11-30 18:32:14.148513: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:14.164213: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmpgk1g058t
2025-11-

## Using 2 digits

In [76]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("4digit_predict_seq_after_2.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features
digit1_data = feature_X['digit1'].values.astype(int)
digit2_data = feature_X['digit2'].values.astype(int)

X_digit1_ohe = to_categorical(digit1_data, num_classes=NUM_DIGITS) # 10 features
X_digit2_ohe = to_categorical(digit2_data, num_classes=NUM_DIGITS) # 10 features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1', 'digit2']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw) # 24 sensor features (12 * 2 snapshots)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (10 + 10 + 24) = 44 features
X = np.concatenate([X_digit1_ohe, X_digit2_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 44
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (20 OHE + 24 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_4digit_seq_after_2.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 4-digit sequence after 2nd digit saved to {tflite_path}")

Dataset loaded: (2525, 27)
Final Model Input Shape: 44 (20 OHE + 24 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 1ms/step - loss: 2.2885 - accuracy: 0.4510 - val_loss: 5.7806 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 710us/step - loss: 0.6564 - accuracy: 0.9173 - val_loss: 8.0768 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 714us/step - loss: 0.3507 - accuracy: 0.9718 - val_loss: 9.2093 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 696us/step - loss: 0.2762 - accuracy: 0.9896 - val_loss: 9.7908 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 693us/step - loss: 0.2391 - accuracy: 0.9941 - val_loss: 10.1876 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 692us/step - loss: 0.2212 - accuracy: 0.9941 - val_loss: 10.3959 - val_accuracy: 0.0000e+00
Epoc

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp1cofuie3/assets


TFLite model for 4-digit sequence after 2nd digit saved to tflite_4digit_seq_after_2.tflite


2025-11-30 18:32:15.609250: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:15.609267: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:15.609346: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp1cofuie3
2025-11-30 18:32:15.609878: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:15.609882: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp1cofuie3
2025-11-30 18:32:15.611087: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:15.626417: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp1cofuie3
2025-11-

## Using 3rd Digit

In [77]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("4digit_predict_seq_after_3.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features
digit1_data = feature_X['digit1'].values.astype(int)
digit2_data = feature_X['digit2'].values.astype(int)
digit3_data = feature_X['digit3'].values.astype(int)

X_digit1_ohe = to_categorical(digit1_data, num_classes=NUM_DIGITS) # 10 features
X_digit2_ohe = to_categorical(digit2_data, num_classes=NUM_DIGITS) # 10 features
X_digit3_ohe = to_categorical(digit3_data, num_classes=NUM_DIGITS) # 10 features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1', 'digit2', 'digit3']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw) # 36 sensor features (12 * 3 snapshots)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (10 + 10 + 10 + 36) = 66 features
X = np.concatenate([X_digit1_ohe, X_digit2_ohe, X_digit3_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 66
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (30 OHE + 36 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_4digit_seq_after_3.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for 4-digit sequence after 3rd digit saved to {tflite_path}")

Dataset loaded: (2525, 40)
Final Model Input Shape: 66 (30 OHE + 36 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 985us/step - loss: 1.9251 - accuracy: 0.5589 - val_loss: 6.7378 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 716us/step - loss: 0.4625 - accuracy: 0.9619 - val_loss: 8.3753 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 708us/step - loss: 0.3000 - accuracy: 0.9866 - val_loss: 8.8890 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 693us/step - loss: 0.2524 - accuracy: 0.9916 - val_loss: 9.3148 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 694us/step - loss: 0.2250 - accuracy: 0.9960 - val_loss: 9.6093 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 689us/step - loss: 0.2105 - accuracy: 0.9970 - val_loss: 9.8046 - val_accuracy: 0.0000e+00
Epoc

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp4z9jdcrs/assets


TFLite model for 4-digit sequence after 3rd digit saved to tflite_4digit_seq_after_3.tflite


2025-11-30 18:32:17.063731: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:17.063747: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:17.063827: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp4z9jdcrs
2025-11-30 18:32:17.064352: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:17.064355: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp4z9jdcrs
2025-11-30 18:32:17.065554: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:17.081016: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp4z9jdcrs
2025-11-

## Using 4th Digit

In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import os
import sys

# --- Configuration ---
NUM_DIGITS = 10 # 0 through 9
RANDOM_SEED = 42
DROPOUT_RATE = 0.3
L2_RATE = 0.001
EPOCHS = 100 
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2
# --- End Configuration ---

# Set random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------
## Load Dataset
# ------------------------
df = pd.read_csv("4digit_predict_seq_after_4.csv")
print("Dataset loaded:", df.shape)

# 1. Separate Features and Target (y)
target_y_raw = df['sequence'].astype(str).values
feature_X = df.drop(columns=["sequence"])

# 2. Extract and One-Hot Encode the Digit Features
digit1_data = feature_X['digit1'].values.astype(int)
digit2_data = feature_X['digit2'].values.astype(int)
digit3_data = feature_X['digit3'].values.astype(int)
digit4_data = feature_X['digit4'].values.astype(int)

# Apply One-Hot Encoding to all four digits separately
X_digit1_ohe = to_categorical(digit1_data, num_classes=NUM_DIGITS) # 10 features
X_digit2_ohe = to_categorical(digit2_data, num_classes=NUM_DIGITS) # 10 features
X_digit3_ohe = to_categorical(digit3_data, num_classes=NUM_DIGITS) # 10 features
X_digit4_ohe = to_categorical(digit4_data, num_classes=NUM_DIGITS) # 10 features

# 3. Extract and Scale Sensor Features
X_sensors_raw = feature_X.drop(columns=['digit1', 'digit2', 'digit3', 'digit4']).values.astype(np.float32)

# Scale sensor readings
scaler = StandardScaler()
X_sensors_scaled = scaler.fit_transform(X_sensors_raw) # 48 sensor features (12 * 4 snapshots)

# 4. Concatenate OHE digits and Scaled Sensors
# Final feature vector X now has (40 OHE + 48 Sensors) = 88 features
X = np.concatenate([X_digit1_ohe, X_digit2_ohe, X_digit3_ohe, X_digit4_ohe, X_sensors_scaled], axis=1)


# Target Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(target_y_raw)
y = to_categorical(y_encoded)


# ------------------------
## Build Model (with Regularization)
# ------------------------
input_shape = X.shape[1]
num_classes = y.shape[1]

model = Sequential([
    tf.keras.layers.Input(shape=(input_shape,)), # Input size 88
    
    # Hidden Layer 1: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        128, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Hidden Layer 2: Dropout and L2 Regularization added
    tf.keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=regularizers.l2(L2_RATE)
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE), 
    
    # Output Layer (Predicts sequence ID)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ------------------------
## Train Model (with Early Stopping)
# ------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

print(f"Final Model Input Shape: {input_shape} (40 OHE + 48 Sensors)")
print(f"Model Output Classes: {num_classes}")

model.fit(
    X, y, 
    validation_split=VALIDATION_SPLIT, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1,
    callbacks=[early_stop]
)

# ------------------------
## Save TFLite Model
# ------------------------
os.makedirs("models_tflite", exist_ok=True)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_path = "tflite_4digit_seq_after_4.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model for full 4-digit sequence saved to {tflite_path}")

Dataset loaded: (2525, 53)
Final Model Input Shape: 88 (40 OHE + 48 Sensors)
Model Output Classes: 25
Epoch 1/100
127/127 [==============================] - 0s 1ms/step - loss: 1.9040 - accuracy: 0.5896 - val_loss: 6.1138 - val_accuracy: 0.0000e+00
Epoch 2/100
127/127 [==============================] - 0s 730us/step - loss: 0.4177 - accuracy: 0.9698 - val_loss: 7.9702 - val_accuracy: 0.0000e+00
Epoch 3/100
127/127 [==============================] - 0s 715us/step - loss: 0.2885 - accuracy: 0.9881 - val_loss: 8.8747 - val_accuracy: 0.0000e+00
Epoch 4/100
127/127 [==============================] - 0s 711us/step - loss: 0.2430 - accuracy: 0.9960 - val_loss: 9.2321 - val_accuracy: 0.0000e+00
Epoch 5/100
127/127 [==============================] - 0s 695us/step - loss: 0.2229 - accuracy: 0.9975 - val_loss: 9.6606 - val_accuracy: 0.0000e+00
Epoch 6/100
127/127 [==============================] - 0s 706us/step - loss: 0.2061 - accuracy: 0.9980 - val_loss: 9.5843 - val_accuracy: 0.0000e+00
Epoch 

INFO:tensorflow:Assets written to: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6jqg4hwv/assets


TFLite model for full 4-digit sequence saved to tflite_4digit_seq_after_4.tflite


2025-11-30 18:32:18.684241: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-30 18:32:18.684259: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-30 18:32:18.684345: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6jqg4hwv
2025-11-30 18:32:18.684820: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-30 18:32:18.684824: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6jqg4hwv
2025-11-30 18:32:18.686030: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-30 18:32:18.701316: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/wr/3v7jt71j699_187nhslrs4wh0000gt/T/tmp6jqg4hwv
2025-11-